# Predição do nível de pobreza doméstica na Costa Rica.

Foi utilizado o conjunto de dados DF, com 9557 linhas e 128 colunas. Cada linha representa uma pessoa e cada coluna um recurso, exclusivo para a família do pessoa ou para a própria pessoa. O conjunto de dados tem a coluna target, que representa o nível de pobreza em uma escala de 1 a 4, sendo que o nível 1 é a pobreza mais extrema. O objetivo desse trabalho é prever o nível de pobreza.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

## Pré processamento

In [ ]:
df = pd.read_csv("/kaggle/input/costa-rican-household-poverty-prediction/train.csv")

In [ ]:
df.head()

In [ ]:
df.drop(['Id','idhogar','r4t3','tamhog','tamviv','hogar_total', 'SQBmeaned', 'SQBhogar_total',
            'SQBage','SQBescolari','SQBedjefe','SQBhogar_nin','SQBovercrowding','SQBdependency',
            'SQBmeaned','agesq'], inplace = True, axis=1)

In [ ]:
df.columns[df.isnull().sum()!=0]

In [ ]:
df['v2a1'] = df['v2a1'].fillna((df['v2a1'].mean()))
df['v18q1'] = df['v18q1'].fillna((df['v18q1'].mean()))
df['rez_esc'] = df['rez_esc'].fillna((df['rez_esc'].mean()))
df['meaneduc'] = df['meaneduc'].fillna((df['meaneduc'].mean()))

df.columns[df.isnull().sum()!=0]

In [ ]:
df.select_dtypes('object').head()

In [ ]:
yes_ou_no = {'no':0,'yes':1}
df['dependency'] = df['dependency'].replace(yes_ou_no).astype(np.float32)
df['edjefe'] = df['edjefe'].replace(yes_ou_no).astype(np.float32)
df['edjefa'] = df['edjefa'].replace(yes_ou_no).astype(np.float32)

In [ ]:
df.select_dtypes('object').head()

## Criando o modelo e usando o Random Forest Classifier

In [ ]:
X = df.drop('Target', axis = 1)

y = df.Target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, max_features=2, oob_score=True, random_state=42)
model.fit(X_train,y_train)


In [ ]:
predicted = model.predict(X_test)
print('Classifcation report:\n', classification_report(y_test, predicted))
print('Confusion matrix:\n', confusion_matrix(y_test, predicted))

Utilizando a matriz de confusão podemos ter as seguintes informações:
- O modelo explicou 85% dos dados.
- 96% dos que o modelo classificou como 1 (extrema pobreza) realmente era. 90% dos que o modelo classificou como 2 realmente era, 95% dos que o modelo classificou como 3 realmente era e 83% dos que o modelo classificou como 4 realmente era.
- O recall explica a frequência que o modelo classifica os dados na classe certa. Podemos ver que na extrema pobreza, o modelo classificaria 66% dos dados que são extrema pobreza como sendo extrema pobreza. É possível ver que a melhor classificação é no 4, pois depois classifica 99% dos dados da classe 4 como sendo classe 4. 
- O F1 score é o equilibrio entre o precision e o recall, o melhor entre as classes 1 a 4, foi a classe 4, que chegou a 91%, isso pode ter acontecido porque a classe 4 é mais frequente no data frame, com 1799 linhas. 

# Tratando os dados desbalanceados

In [ ]:
df['Target'].value_counts()

In [ ]:
df_1 = df[df['Target']== 1]
df_2 = df[df['Target']==2]
df_3 = df[df['Target']==3]
df_4 = df[df['Target']== 4]
df_1.shape,df_2.shape,df_3.shape,df_4.shape

In [ ]:
from sklearn.utils import resample

df_1_over = resample(df_1,
                       replace=True, # sample com replacement
                       n_samples=len(df_4), # igualando a maior classe
                       random_state=42)
df_2_over = resample(df_2,
                       replace=True, # sample com replacement
                       n_samples=len(df_4), # igualando a maior classe
                       random_state=42)
df_3_over = resample(df_3,
                       replace=True, # sample com replacement
                       n_samples=len(df_4), # igualando a maior classe
                       random_state=42)

In [ ]:
df_over = pd.concat([df_1_over,df_2_over,df_3_over,df_4])

In [ ]:
df_over['Target'].value_counts()


# Refazendo o modelo

In [ ]:
X = df_over.drop('Target', axis = 1)

y = df_over.Target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, max_features=2, oob_score=True, random_state=42)
model.fit(X_train,y_train)


In [ ]:
predicted = model.predict(X_test)
print('Classifcation report:\n', classification_report(y_test, predicted))
print('Confusion matrix:\n', confusion_matrix(y_test, predicted))

Refazendo o modelo com os dados balanceados, é possivel ver uma melhora significativa, pois agora o modelo explica 99% dos dados e houve uma melhora do precision, recall e f1 score nas classes 1 a 3

In [ ]:
df_test = pd.read_csv("/kaggle/input/costa-rican-household-poverty-prediction/test.csv")
z = pd.Series(predicted,name="Target")
df_entrega = pd.concat([df_test.Id,z], axis=1)
df_entrega.to_csv("/kaggle/working/submission.csv",index=False)